In [67]:
import sqlite3
print(sqlite3.version)
print(sqlite3.sqlite_version)

2.6.0
3.38.2


In [76]:
conn = sqlite3.connect("chinook.db")
cur = conn.cursor()

albums = cur.execute("""select * from albums order by title limit 3; """).fetchall()
print(albums)

[(156, '...And Justice For All', 50), (257, '20th Century Masters - The Millennium Collection: The Best of Scorpions', 179), (296, 'A Copland Celebration, Vol. I', 230)]


In [30]:
type(albums)

list

In [31]:
cur.execute("""select * from albums order by title limit 3; """).fetchone()

(156, '...And Justice For All', 50)

In [32]:
albumdata = cur.execute("""select * from albums order by title limit 3; """).fetchmany(5)

In [33]:
albumdata

[(156, '...And Justice For All', 50),
 (257,
  '20th Century Masters - The Millennium Collection: The Best of Scorpions',
  179),
 (296, 'A Copland Celebration, Vol. I', 230)]

In [62]:
#convert the list returned to a dataframe..


from pandas import DataFrame



DataFrame(albumdata)
#and if you want assign enything into this data frame just make it like this
#df = DataFrame(albumdata)


,0,1,2
0,156,...And Justice For All,50
1,257,20th Century Masters - The Millennium Collecti...,179
2,296,"A Copland Celebration, Vol. I",230


In [63]:
import sqlite3 
import pandas as pd

# return query results as a dataframe

dfalbum = pd.read_sql_query("SELECT * FROM albums", conn)
dfalbum.head()

,AlbumId,Title,ArtistId
0,1,For Those About To Rock We Salute You,1
1,2,Balls to the Wall,2
2,3,Restless and Wild,2
3,4,Let There Be Rock,1
4,5,Big Ones,3


# SQLite has a database catalog

In [36]:
pd.read_sql_query("select * from sqlite_master where type = 'table';", conn)

,type,name,tbl_name,rootpage,sql
0,table,albums,albums,2,"CREATE TABLE ""albums""\r\n(\r\n [AlbumId] IN..."
1,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
2,table,artists,artists,4,"CREATE TABLE ""artists""\r\n(\r\n [ArtistId] ..."
3,table,customers,customers,5,"CREATE TABLE ""customers""\r\n(\r\n [Customer..."
4,table,employees,employees,8,"CREATE TABLE ""employees""\r\n(\r\n [Employee..."
5,table,genres,genres,10,"CREATE TABLE ""genres""\r\n(\r\n [GenreId] IN..."
6,table,invoices,invoices,11,"CREATE TABLE ""invoices""\r\n(\r\n [InvoiceId..."
7,table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items""\r\n(\r\n [Invo..."
8,table,media_types,media_types,15,"CREATE TABLE ""media_types""\r\n(\r\n [MediaT..."
9,table,playlists,playlists,16,"CREATE TABLE ""playlists""\r\n(\r\n [Playlist..."


In [37]:
pd.read_sql_query("select  name from sqlite_master where type = 'index';", conn)

,name
0,sqlite_autoindex_playlist_track_1
1,IFK_AlbumArtistId
2,IFK_CustomerSupportRepId
3,IFK_EmployeeReportsTo
4,IFK_InvoiceCustomerId
5,IFK_InvoiceLineInvoiceId
6,IFK_InvoiceLineTrackId
7,IFK_PlaylistTrackTrackId
8,IFK_TrackAlbumId
9,IFK_TrackGenreId


In [38]:
dftablist = pd.read_sql_query("select name from sqlite_master where type = 'table';", conn)

In [39]:
dftablist.head(3)

,name
0,albums
1,sqlite_sequence
2,artists


In [40]:
# GET TABLE Schema....

In [41]:
pd.read_sql_query("""
PRAGMA table_info('albums');
""", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,AlbumId,INTEGER,1,None,1
1,1,Title,NVARCHAR(160),1,None,0
2,2,ArtistId,INTEGER,1,None,0


# Suppressing the default index....

In [42]:
qry = '''
select substr(a.name, 1, 10) as Artist,
       substr(title,1, 15) as Album,
       t.name as Song
from albums r
join artists a
on (r.artistid = a.artistid)
join tracks t
on (r.albumid = t.albumid)
order by a.name, title
limit 15;
'''

pd.read_sql_query(qry, conn, index_col = 'Artist') #index_col replaces the index

,Album,Song
Artist,,
AC/DC,For Those About,For Those About To Rock (We Salute You)
AC/DC,For Those About,Put The Finger On You
AC/DC,For Those About,Let's Get It Up
AC/DC,For Those About,Inject The Venom
AC/DC,For Those About,Snowballed
AC/DC,For Those About,Evil Walks
AC/DC,For Those About,C.O.D.
AC/DC,For Those About,Breaking The Rules
AC/DC,For Those About,Night Of The Long Knives


# Let's create a new table from a query

In [ ]:
# Note The create table statment before the select query....

qry = '''
create table if not exists top15 as
select substr(a.name, 1, 10) as Artist,
       substr(title,1, 15) as Album,
       t.name as Song
from albums r
join artists a 
on (r.ArtistId = a.ArtistId)
join tracks t
on (r.AlbumId = t.AlbumId)
order by a.name, title
limit 15;
'''


albums = cur.execute(qry)

In [ ]:
pd.read_sql_query('select * from top15', conn)

In [ ]:
# but They All not working i dont know

# Let's Close The connection 

In [79]:
cur.close()
conn.close()

# Connecting to PostgreSQL

In [ ]:
pip install -U psycopg2